In [31]:
import hls4ml
import onnxruntime as ort
import onnx
import plotting

In [53]:
TARGET_FPGA_DEVICE = "XCZU7EV-2FFVC1156"
ONNX_PATH = "onnx/tooth.onnx"
model = onnx.load(ONNX_PATH)
onnx.checker.check_model(model)

In [54]:
# First, the baseline model
hls_config = hls4ml.utils.config_from_onnx_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['Model']['ReuseFactor'] = 1

for Layer in hls_config['LayerName'].keys():
  hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
  hls_config['LayerName'][Layer]['ReuseFactor'] = 1
hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'
plotting.print_dict(hls_config)
for k in hls_config:
  print('\n',k,hls_config[k],'\n')

Model
  Precision:         ap_fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency

 Model {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'} 



In [ ]:
import json
print(json.dumps(hls_config, indent = 4)) #chipSkywalker 2024.07.04 to fix no attribute print_dict error

#cfg = hls4ml.converters.create_config(backend='Vivado')
cfg = hls4ml.converters.create_config(backend='Vitis')  # chipSkywalker modified to use Vitis HLS
cfg['IOType'] = 'io_stream'  # Must set this if using CNNs!
cfg['HLSConfig'] = hls_config
cfg['OnnxModel'] = model
cfg['OutputDir'] = 'yolo/'
cfg['Part'] = TARGET_FPGA_DEVICE   # chipSkywalker use Target Device set at beginning
#cfg['XilinxPart'] = TARGET_FPGA_DEVICE   
#cfg['XilinxPart'] = 'xcu250-figd2104-2L-e'  
hls_model = hls4ml.converters.onnx_to_hls(cfg) ## Unsupported operation type: QuantizeLinear
hls_model.compile()